In [ ]:
import networkx as nx
import pickle
import matplotlib.pyplot as plt
from networkx.algorithms import isomorphism
import random

In [ ]:
#Load the Dataset
data = pickle.load(open('labeled_data\\fully_connected_network.pkl', 'rb'))

In [ ]:
data.nodes[1]

In [ ]:
# Creating the structure of the tasks
project = nx.Graph()
node_list = ["java", "cpp", "web"]
edge_list = [("java", "web"),("cpp", "web")]
project.add_nodes_from(node_list)
project.add_edges_from(edge_list)

In [ ]:
# Initialize Sets for each Team
Team_java = set()
Team_cpp = set()
Team_web = set()

Task_Java_skills = set(['Mobile App Development', 'Android', 'iPhone', 'Java',
                       'Linux', 'System Admin', 'Flutter', 'HTML', 'Pine Script', 'eCommerce', 'Swift',
                        'Web Scraping', 'Flutter', 'ASP.NET', 'MySQL', 'API', 'Amazon Web Services',
                        'Android Studio', 'API Integration'])
Task_Cpp_skills = set(['C++ Programming', 'C Programming', 'C# Programming',
                      'Visual Basic', 'Software Architecture', 'PHP', 'Linux', 'JavaScript',
                       'Blockchain', 'API', 'Engineering', 'MySQL', '.NET', 'API Integration'])
Task_Web_skills = set(['SEO', 'Website Design', 'PHP', 'WordPress', 'HTML', 'Website Design',
                      'JavaScript', 'HTML5', 'PHP', 'JavaScript', 'React.js', 'Graphic Design',
                       'CSS', 'Logo Design', 'Photoshop', 'Illustrator', 'eCommerce', 'Prestashop',
                       'PSD to HTML''Content Marketing', 'API', 'Python', 'Web Hosting', 'PSD to HTML',
                       'Advertising', 'Shopify', 'App Developer', 'Django'])

Covered_Java_skills = set()
Covered_Cpp_skills = set()
Covered_Web_skills = set()

In [ ]:
# Create teams
# Define the attribute to filter by
attribute_to_filter = 'label'
java_value = 'java'
cpp_value = 'cpp'
web_value = 'web'

# Filter nodes based on the attribute
filtered_java_nodes = [node for node, data in data.nodes(data=True) if data.get(attribute_to_filter) == java_value]
filtered_cpp_nodes = [node for node, data in data.nodes(data=True) if data.get(attribute_to_filter) == cpp_value]
filtered_web_nodes = [node for node, data in data.nodes(data=True) if data.get(attribute_to_filter) == web_value]

# Create a subgraph with the filtered nodes
java_network = data.subgraph(filtered_java_nodes)
cpp_network = data.subgraph(filtered_cpp_nodes)
web_network = data.subgraph(filtered_web_nodes)

In [ ]:
def greedyCover(network, skills):
    covered_skills = set()
    network_copy = network.copy()
    selected_set = set()

    while len(skills-covered_skills) > 0:
        max_cover = set()
        top_node = None
        for node in network_copy.nodes:
            local_skills = set(network_copy.nodes[node]['skills'])
            # print(f"Node Skills --> {local_skills}")
            if len(local_skills.intersection(skills-covered_skills)) > len(max_cover):
                max_cover = local_skills
                top_node = node
        # print(f"Best Skill set --> {max_cover}")
        if top_node is not None:
            selected_set.add(top_node)
            covered_skills = covered_skills.union(max_cover)
            network_copy.remove_node(top_node)
        else:
            break
        # print(f"Covered Skills = {covered_skills}")
    return list(selected_set)

In [ ]:
cpp_best = greedyCover(cpp_network, Task_Cpp_skills)
java_best = greedyCover(java_network, Task_Java_skills)
web_best = greedyCover(web_network, Task_Web_skills)

best = []
best.extend(cpp_best)
best.extend(java_best)
best.extend(web_best)

best_in_network = data.subgraph(best)

java_best_network = data.subgraph(java_best)
cpp_best_network = data.subgraph(cpp_best)
web_best_network = data.subgraph(web_best)

new_network = best_in_network.copy()

for u in java_best_network.nodes:
    for v in cpp_best_network.nodes:
        new_network.remove_edge(u, v)

In [ ]:
def isIsomorphic(G, H):
    GM = isomorphism.GraphMatcher(G, H)
    if GM.is_isomorphic():
        # Extract the nodes of the subgraph from G
        subgraph_nodes = list(GM.mapping.keys())
        subgraph = G.subgraph(subgraph_nodes)
        
        print("Subgraph with similar structure found:")
        # print(subgraph.edges())
        return True
    else:
        print("No subgraph with a similar structure found.")
        return False

In [ ]:
def getConnectedComponents(G):

    # Find the connected components in the graph
    connected_components = list(nx.connected_components(G))

    # Find the largest connected component
    largest_component = max(connected_components, key=len)

    # Create a subgraph containing only the nodes in the largest connected component
    subgraph = G.subgraph(largest_component)

    return subgraph

In [ ]:
def getSubgraph(G, H, w):
    list_of_nodes = list(G.nodes)
    random.shuffle(list_of_nodes)
    print(f"Shuffled Nodes: {list_of_nodes}")
    selected_nodes = []
    labels = []
    i = 0
    j = 0
    try:
        while i < len(H.nodes) and j < len(list_of_nodes):
            if (G.nodes[list_of_nodes[j]]['label'] not in labels) and (G.nodes[list_of_nodes[j]] not in selected_nodes):
                selected_nodes.append(list_of_nodes[j])
                labels.append(G.nodes[list_of_nodes[j]]['label'])
                print(f"Added Node {i+1}")
                i += 1
            j += 1
    except: 
        print("Error Getting Isomorphic Subgraph")

    subGraph = G.subgraph(selected_nodes)
    if isIsomorphic(subGraph, H):
        print(subGraph.nodes)
        return subGraph
    else:
        while w > 0:
            getSubgraph(G, H, w)
            w -= 1

In [ ]:
try_network = new_network.subgraph([1, 905, 1565])

In [ ]:
getSubgraph(try_network, project, 3)

In [ ]:
def guessGreedy(G, H, w):
    # test_subgraph = 
    while G.number_of_edges() >= H.number_of_edges():
        network = getConnectedComponents(G)
        print(f"Selected network {network}")
        subgraph = getSubgraph(network, H, w)
        print(f"subgraph found {subgraph.nodes}")
        max_weight_edge = max(subgraph.edges(data=True), key=lambda edge: edge[2]['weight'])
        k = max_weight_edge[0]
        l = max_weight_edge[1]
        # print("I got this far!")
        for (u, v) in G.edges():
            # print("\nI mean this far")
            if (G.nodes[u]['label'] == G.nodes[l]['label']) and (G.nodes[v]['label'] == G.nodes[k]['label']) and G[u][v]['weight'] > max_weight_edge[2]['weight']:
                G.remove_edge(u, v)
                print("Edge Removed")
        print(f"Nodes remaining in G: {getConnectedComponents(G)}")
    return getConnectedComponents(G)

In [ ]:
guessGreedy(new_network, project, 3)